In [64]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

from imblearn.metrics import geometric_mean_score
from sklearn.metrics import balanced_accuracy_score
import statistics as stats

# Se importan las librerías de AdaBoost y de árboles de decisión 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from imblearn.metrics import geometric_mean_score
from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import RUSBoostClassifier

from imblearn.datasets import fetch_datasets

import math

import random
import array

from deap import base
from deap import creator
from deap import tools

In [65]:
from collections import Counter

import numpy as np
from sklearn.base import is_regressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble.forest import BaseForest
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.tree.tree import BaseDecisionTree
from sklearn.utils import check_random_state
from sklearn.utils import check_X_y
#from sklearn.utils import shuffle

In [66]:
class SMOTE(object):
    """Implementation of Synthetic Minority Over-Sampling Technique (SMOTE).
    SMOTE performs oversampling of the minority class by picking target 
    minority class samples and their nearest minority class neighbors and 
    generating new samples that linearly combine features of each target 
    sample with features of its selected minority class neighbors [1].
    Parameters
    ----------
    k_neighbors : int, optional (default=5)
        Number of nearest neighbors.
    random_state : int or None, optional (default=None)
        If int, random_state is the seed used by the random number generator.
        If None, the random number generator is the RandomState instance used
        by np.random.
    References
    ----------
    .. [1] N. V. Chawla, K. W. Bowyer, L. O. Hall, and P. Kegelmeyer. "SMOTE:
           Synthetic Minority Over-Sampling Technique." Journal of Artificial
           Intelligence Research (JAIR), 2002.
    """

    def __init__(self, k_neighbors=5, random_state=None):
        self.k = k_neighbors
        self.random_state = random_state

    def sample(self, n_samples):
        """Generate samples.
        Parameters
        ----------
        n_samples : int
            Number of new synthetic samples.
        Returns
        -------
        S : array, shape = [n_samples, n_features]
            Returns synthetic samples.
        """
        np.random.seed(seed=self.random_state)

        S = np.zeros(shape=(n_samples, self.n_features))
        # Calculate synthetic samples.
        for i in range(n_samples):
            j = np.random.randint(0, self.X.shape[0])

            # Find the NN for each sample.
            # Exclude the sample itself.
            nn = self.neigh.kneighbors(self.X[j].reshape(1, -1),
                                       return_distance=False)[:, 1:]
            nn_index = np.random.choice(nn[0])

            dif = self.X[nn_index] - self.X[j]
            gap = np.random.random()

            S[i, :] = self.X[j, :] + gap * dif[:]

        return S

    def fit(self, X):
        """Train model based on input data.
        Parameters
        ----------
        X : array-like, shape = [n_minority_samples, n_features]
            Holds the minority samples.
        """
        self.X = X
        self.n_minority_samples, self.n_features = self.X.shape

        # Learn nearest neighbors.
        self.neigh = NearestNeighbors(n_neighbors=self.k + 1)
        self.neigh.fit(self.X)

        return self

In [67]:
#classifies sample x to the class which center is closer to
def classify(x,centers):
    dist_majcenter = euclidean(x,centers[:len(x)])
    dist_mincenter = euclidean(x,centers[len(x):])
    return np.argmin([dist_majcenter,dist_mincenter])

# function to generate the initial random population from the training set
def load_individuals(X,y,maj_class,min_class,creator,n):
    """

    """
    maj_samples = X[y == maj_class]
    min_samples = X[y == min_class]
    individuals = []
    for i in range(n):
        random_maj = maj_samples[random.randint(0,maj_samples.shape[0]-1)]
        random_min = min_samples[random.randint(0,min_samples.shape[0]-1)]
        individual = np.asarray(np.concatenate((random_maj,random_min)))

        individual = creator(individual)
        individuals.append(individual)
    return individuals

# returns the euclidean distance between two points
def euclidean(v1, v2):
    return sum((p-q)**2 for p, q in zip(v1, v2)) ** .5

#returns the sum of the distances from each sample in X_train to the closest center
#we are interested in minimizing this sum of distances
def evaluate(X,individual):
    D = X.shape[1]
    S = 0
    for x in X:
        dist = dist_to_closest_center(x,individual[:D],individual[D:])
        S += dist

    return S,

#computes the euclidean distance for both centers and returns the shortest one
def dist_to_closest_center(x,maj_center,min_center):
    dist_majcenter = euclidean(x,maj_center)
    dist_mincenter = euclidean(x,min_center)
    return min(dist_majcenter,dist_mincenter)

In [68]:
class DEClustering(object):
    """
    DOCUMENTARRRRR
    """
    def __init__(self, X_train, y_train, maj_class, min_class):
        
        self.X = X_train
        self.y = y_train
        self.maj_class = maj_class
        self.min_class = min_class
        self.NDIM = self.X.shape[1]
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)
        self.toolbox = base.Toolbox()
        self.toolbox.register("population",load_individuals, self.X, self.y, self.maj_class, self.min_class,
                              creator.Individual)
        self.toolbox.register("select", tools.selRandom, k=3)
        self.toolbox.register("evaluate", evaluate, self.X)
        
    def fit(self, CR, F, POP_SIZE, NGEN):
        # Differential evolution parameters
        #CR = 0.25
        #F = 1  
        #MU = 300
        #NGEN = 200    

        pop = self.toolbox.population(n=POP_SIZE);
        hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "std", "min", "avg", "max"

        # Evaluate the individuals
        fitnesses = self.toolbox.map(self.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(pop), **record)
#         print(logbook.stream)
        
        last_fitness = float('inf')
        for g in range(1, NGEN):
            for k, agent in enumerate(pop):
                a,b,c = self.toolbox.select(pop)
                y = self.toolbox.clone(agent)
                index = random.randrange(self.NDIM)
                for i, value in enumerate(agent):
                    if i == index or random.random() < CR:
                        y[i] = a[i] + F*(b[i]-c[i])
                y.fitness.values = self.toolbox.evaluate(y)
                if y.fitness > agent.fitness:
                    pop[k] = y
                #print(pop[k].fitness)
                
#                 if abs(last_fitness-agent.fitness.values[0])<1:
#                     g = NGEN
#                 last_fitness = agent.fitness.values[0]
            hof.update(pop)
            record = stats.compile(pop)
            logbook.record(gen=g, evals=len(pop), **record)
#             print(logbook.stream)

#         print("Best individual is ", hof[0], hof[0].fitness.values[0])
#         return hof[0]
        self.best_ind = hof[0]
        self.best_fitness = hof[0].fitness.values[0]
#         print(self.best_ind)
#         print(self.best_fitness)
        return self

In [69]:
class DECLUndersampling(object):
    def __init__(self, X_train, y_train, maj_class, min_class, H=6, CR=0.6, F=0.5, POP_SIZE=10, NGEN=100, alpha=0.8):
        self.X = X_train
        self.y = y_train
        self.maj_class = maj_class
        self.min_class = min_class
        self.classes = [self.maj_class,self.min_class]
        self.H = H
        self.CR = CR
        self.F = F
        self.POP_SIZE = POP_SIZE
        self.NGEN = NGEN
        self.alpha = alpha
        self.decl = DEClustering(self.X,self.y,self.maj_class,self.min_class)
#         super(DECLUndersampling, self).__init__(X=X_train, y=y_train)
    
    def undersample(self):
        #compute H clustering processes
        clustering_centers = []
        for i in range(int(self.H)):
            self.decl.fit(self.CR,self.F,self.POP_SIZE,self.NGEN)
            centers = self.decl.best_ind
            clustering_centers.append(centers)
        
        #compute cluster stability for each majority sample
        majority_samples = self.X[self.y==self.maj_class]
        cluster_stabilities = []
        for sample in majority_samples:
            S = 0
            for clustering in clustering_centers:
                c = self.classes[classify(sample,clustering)]
                if c==self.maj_class:
                    S += 1
            cluster_stabilities.append(S/self.H)
            
        #compute boundary and non-boundary samples
        boundary_points = majority_samples[np.array(cluster_stabilities)<=self.alpha]
        non_boundary_points = majority_samples[np.array(cluster_stabilities)>self.alpha]
        
        #undersample non-boundary samples
        RUSsize = int(non_boundary_points.shape[0]*0.4)
        #RUS PURO
        indices = np.random.randint(non_boundary_points.shape[0], size=RUSsize)
        nbp_us = non_boundary_points[indices]
#         #RUS ponderado por el cluster stability de cada ejemplo
#         C_non_boundary = np.array(cluster_stabilities)[np.array(cluster_stabilities)>self.alpha]
#         indices = np.random.choice(np.arange(non_boundary_points.shape[0]),size=RUSsize,
#                                    p = C_non_boundary/sum(C_non_boundary))
#         nbp_us = non_boundary_points[indices]
        
        #build undersampled training set
        new_majorityclass_training = np.vstack((boundary_points,nbp_us))
        print("Conjunto de entrenamiento original de tamaño: {}".format(self.X.shape[0]))
        n_may,n_min = sum(self.y == self.maj_class),sum(self.y == self.min_class)
        print("De los cuales:\n \t nº de ejemplos clase MAYORITARIA: {}\n \t nº de ejemplos clase MINORITARIA: {}"
              .format(n_may,n_min))
#         print("CONJUNTO DE DATOS NO-BALANCEADO")
#         print("IR = {}".format(n_may/n_min))
        minority_samples = self.X[self.y==self.min_class]
        #prepare adaboost training set joining the undersampled majority class instances with the minority class instances
        self.X_US = np.vstack((new_majorityclass_training,minority_samples))
        self.y_US = np.hstack((np.full(new_majorityclass_training.shape[0],self.maj_class),
                          np.full(minority_samples.shape[0],self.min_class)))


        print("nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: {}"
              .format(new_majorityclass_training.shape[0]))
        print("Conjunto de entrenamiento actual de tamaño: {}".format(new_majorityclass_training.shape[0]+n_min))
        
        return self.X_US, self.y_US

In [70]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from imblearn.metrics import geometric_mean_score

def Gmean(clf,X_test,y_test):
    y_pred = clf.predict(X_test)
    gmean = geometric_mean_score(y_test, y_pred)
    #print("Gmean:",gmean)
    return gmean
    
def compute_fitness(p,X_before_SMOTE,y_before_SMOTE,maj_class,min_class,synthetic_samples, individual):
    #print(synthetic_samples[0])
    selected_syn = []
    for i, value in enumerate(individual):
        if individual[i]>0:
            selected_syn.append(synthetic_samples[i-1])
    #selected_syn = synthetic_samples[individual>0]
    selected_syn = np.array(selected_syn)
    X = np.vstack((X_before_SMOTE,selected_syn))
    y = np.hstack((y_before_SMOTE,np.full(selected_syn.shape[0],min_class)))
    #realizar splitTest??????????????''
    Xtr, Xtst, ytr, ytst = train_test_split(X, y, test_size=0.3)
    #dt = trainDT(Xtr,ytr,weights)
    dt = trainDT(Xtr,ytr)
    #test??????????????????????????
    #G = Gmean(dt,X_before_SMOTE,y_before_SMOTE)
    G = Gmean(dt,Xtst,ytst)
    n_minority = len(individual)+sum(individual)
    n_majority = X_before_SMOTE[y_before_SMOTE==maj_class].shape[0]
    #f = G - abs(1-(n_minority/n_majority*p))
    #print("f: ",f)
    return G,

def trainDT(X_train,y_train,w=None):
    clf = DecisionTreeClassifier()
    clf = clf.fit(X_train,y_train,sample_weight=w)
    return clf

In [71]:
class DESMOTE(object):
    """
    DOCUMENTARRRRR
    """
    def __init__(self, X_train, y_train, maj_class, min_class,synthetic_samples):
        
        self.X = X_train
        self.y = y_train
        self.maj_class = maj_class
        self.min_class = min_class
        self.syn = synthetic_samples
        
        N = self.X.shape[0]
        minority_samples = self.X[self.y==self.min_class]
        majority_samples = self.X[self.y==self.maj_class]
        n_maj = majority_samples.shape[0]
        n_min = minority_samples.shape[0]
        self.NDIM = n_min
        
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax, clf=None)
        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_int", random.randint, 0, 1)
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_int, n_min)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("select", tools.selRandom, k=3)
        self.toolbox.register("evaluate", compute_fitness, 0.2,
                     self.X,self.y,self.maj_class,self.min_class,self.syn)        
    
    def fit(self, CR, F, POP_SIZE, NGEN):
        # Differential evolution parameters
        #CR = 0.25
        #F = 1  
        #MU = 300
        #NGEN = 200    

        pop = self.toolbox.population(n=POP_SIZE);
        hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "std", "min", "avg", "max"
        #print(pop)
        # Evaluate the individuals
        fitnesses = self.toolbox.map(self.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(pop), **record)
        #print(logbook.stream)

        for g in range(1, NGEN):
            for k, agent in enumerate(pop):
                a,b,c = self.toolbox.select(pop)
                #we adopt a self-adaptative operator
                #l = math.exp(1-(NGEN/(NGEN+1-g)))
                #F = F_0*(2**l)
                #d = toolbox.clone(agent) #donor vector
                #sig_d = toolbox.clone(agent)
                y = self.toolbox.clone(agent)
                index = random.randrange(self.NDIM)
                for i, value in enumerate(agent):
                    #d[i] = a[i] + F*(b[i]-c[i]) #donor vector
                    #the mutated donor is mapped to binary space by a sigmoid function with displacement
                    #sig_d[i] = round(1/(1+math.exp(-(d[i]))))
                    if i == index or random.random() < CR:
                        y[i] = a[i] + F*(b[i]-c[i])
    #                     y[i] = sig_d[i]
                y.fitness.values = self.toolbox.evaluate(y)
                if y.fitness > agent.fitness:
                    pop[k] = y
                #print(pop[k].fitness)
            hof.update(pop)
            record = stats.compile(pop)
            logbook.record(gen=g, evals=len(pop), **record)
            #print(logbook.stream)

        #print("Best individual is ", hof[0], hof[0].fitness.values[0])
#         return hof[0]
        self.best_ind = hof[0]
        self.best_fitness = hof[0].fitness.values[0]
#         print(self.best_ind)
#         print(self.best_fitness)
        return self

In [72]:
class DERSBoost(AdaBoostClassifier):
    """Implementation of DERSBoost.
    DERSBoost introduces data sampling into the AdaBoost algorithm by both
    undersampling the majority class guided by a Differential Evolutionary algorithm, and
    oversampling the minority class using also a DE-guided SMOTE procedure on each boosting iteration [1].
    This implementation inherits methods from the scikit-learn 
    AdaBoostClassifier class, only modifying the `fit` method.
    Parameters
    ----------
    n_samples : int, optional (default=100)
        Number of new synthetic samples per boosting step.
    k_neighbors : int, optional (default=5)
        Number of nearest neighbors.
    base_estimator : object, optional (default=DecisionTreeClassifier)
        The base estimator from which the boosted ensemble is built.
        Support for sample weighting is required, as well as proper `classes_`
        and `n_classes_` attributes.
    n_estimators : int, optional (default=50)
        The maximum number of estimators at which boosting is terminated.
        In case of perfect fit, the learning procedure is stopped early.
    learning_rate : float, optional (default=1.)
        Learning rate shrinks the contribution of each classifier by
        ``learning_rate``. There is a trade-off between ``learning_rate`` and
        ``n_estimators``.
    algorithm : {'SAMME', 'SAMME.R'}, optional (default='SAMME.R')
        If 'SAMME.R' then use the SAMME.R real boosting algorithm.
        ``base_estimator`` must support calculation of class probabilities.
        If 'SAMME' then use the SAMME discrete boosting algorithm.
        The SAMME.R algorithm typically converges faster than SAMME,
        achieving a lower test error with fewer boosting iterations.
    random_state : int or None, optional (default=None)
        If int, random_state is the seed used by the random number generator.
        If None, the random number generator is the RandomState instance used
        by np.random.
    References
    ----------
    .. [1] N. V. Chawla, A. Lazarevic, L. O. Hall, and K. W. Bowyer.
           "SMOTEBoost: Improving Prediction of the Minority Class in
           Boosting." European Conference on Principles of Data Mining and
           Knowledge Discovery (PKDD), 2003.
    """

    def __init__(self,
                 n_samples=100,
                 k_neighbors=5,
                 base_estimator=None,
                 n_estimators=50,
                 learning_rate=1.,
                 algorithm='SAMME.R',
                 random_state=None,
                 maj_class = -1,
                 min_class = 1,
                 H=6,
                 CR=0.6,
                 F=0.5,
                 POP_SIZE=10,
                 NGEN=100,
                 us_alpha=0.8
                ):

        self.n_samples = n_samples
        self.algorithm = algorithm
        self.maj_class = maj_class
        self.min_class = min_class
        self.H = H
        self.CR = CR
        self.F = F
        self.POP_SIZE = POP_SIZE
        self.NGEN = NGEN
        self.us_alpha = us_alpha
        self.smote = SMOTE(k_neighbors=k_neighbors,
                           random_state=random_state)

        super(DERSBoost, self).__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=random_state)
        
    def fit(self, X, y, sample_weight=None, minority_target=None):
        """Build a boosted classifier/regressor from the training set (X, y),
        performing SMOTE during each boosting step.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape = [n_samples, n_features]
            The training input samples. Sparse matrix can be CSC, CSR, COO,
            DOK, or LIL. COO, DOK, and LIL are converted to CSR. The dtype is
            forced to DTYPE from tree._tree if the base classifier of this
            ensemble weighted boosting classifier is a tree or forest.
        y : array-like of shape = [n_samples]
            The target values (class labels in classification, real numbers in
            regression).
        sample_weight : array-like of shape = [n_samples], optional
            Sample weights. If None, the sample weights are initialized to
            1 / n_samples.
        minority_target : int
            Minority class label.
        Returns
        -------
        self : object
            Returns self.
        Notes
        -----
        Based on the scikit-learn v0.18 AdaBoostClassifier and
        BaseWeightBoosting `fit` methods.
        """
        # Check that algorithm is supported.
        if self.algorithm not in ('SAMME', 'SAMME.R'):
            raise ValueError("algorithm %s is not supported" % self.algorithm)

        # Check parameters.
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero")

        if (self.base_estimator is None or
                isinstance(self.base_estimator, (BaseDecisionTree,
                                                 BaseForest))):
            DTYPE = np.float64  # from fast_dict.pxd
            dtype = DTYPE
            accept_sparse = 'csc'
        else:
            dtype = None
            accept_sparse = ['csr', 'csc']

        X, y = check_X_y(X, y, accept_sparse=accept_sparse, dtype=dtype,
                         y_numeric=is_regressor(self))
        
#         #DE-guided clustering
#         cluster_centers = compute_clusters(X,y)
#         cluster_stabilities = compute_cluster_stabilities(X,y,self.maj_class,cluster_centers)
#         #UNDERSAMPLING
#         boundary, nonboundary = compute_boundary_and_nonboundary_samples(X,y,self.maj_class,cluster_stabilities)
#         nonboundary_us = undersample_nonboundary_samples(nonboundary,cluster_stabilities)
#         X, y = undersampled_training_set(X,y,boundary,nonboundary_us,self.maj_class,self.min_class)


        declu = DECLUndersampling(X, y, self.maj_class, self.min_class, CR=self.CR, F=self.F,
                                  POP_SIZE=self.POP_SIZE, NGEN=self.NGEN, alpha=self.us_alpha)
        X, y = declu.undersample()
        
        if sample_weight is None:
            # Initialize weights to 1 / n_samples.
            sample_weight = np.empty(X.shape[0], dtype=np.float64)
            sample_weight[:] = 1. / X.shape[0]
        else:
            sample_weight = check_array(sample_weight, ensure_2d=False)
            # Normalize existing weights.
            sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

            # Check that the sample weights sum is positive.
            if sample_weight.sum() <= 0:
                raise ValueError(
                    "Attempting to fit with a non-positive "
                    "weighted number of samples.")

        if minority_target is None:
            # Determine the minority class label.
            stats_c_ = Counter(y)
            maj_c_ = max(stats_c_, key=stats_c_.get)
            min_c_ = min(stats_c_, key=stats_c_.get)
            self.minority_target = min_c_
        else:
            self.minority_target = minority_target

        # Check parameters.
        self._validate_estimator()

        # Clear any previous fit results.
        self.estimators_ = []
        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float64)
        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float64)

        random_state = check_random_state(self.random_state)

        for iboost in range(self.n_estimators):
            X_min = X[np.where(y == self.minority_target)]

            # SMOTE step.
            if len(X_min) >= self.smote.k:
                self.smote.fit(X_min)
                X_syn = self.smote.sample(n_samples = len(X_min))
#                 y_syn = np.full(X_syn.shape[0], fill_value=self.minority_target,
#                                 dtype=np.int64)

                #DEguided selection of best synthetics
                desmote = DESMOTE(X,y,self.maj_class,self.min_class,X_syn)
                desmote.fit(self.CR,self.F,self.POP_SIZE,self.NGEN)
                selected_syn = []
                for i, value in enumerate(desmote.best_ind):
                    if desmote.best_ind[i]>0:
                        selected_syn.append(X_syn[i])
                selected_syn = np.array(selected_syn)
                y_syn = np.full(selected_syn.shape[0], fill_value=self.minority_target,
                                dtype=np.int64)
                
                # Normalize synthetic sample weights based on current training set.
                sample_weight_syn = np.empty(selected_syn.shape[0], dtype=np.float64)
                sample_weight_syn[:] = 1. / X.shape[0]
                
                # Combine the original and synthetic samples.
                X = np.vstack((X, selected_syn))
                y = np.append(y, y_syn)
                
                # Combine the weights.
                sample_weight = \
                    np.append(sample_weight, sample_weight_syn).reshape(-1, 1)
                sample_weight = \
                    np.squeeze(normalize(sample_weight, axis=0, norm='l1'))

                # X, y, sample_weight = shuffle(X, y, sample_weight,
                #                              random_state=random_state)
                
            # Boosting step.
            sample_weight, estimator_weight, estimator_error = self._boost(
                iboost,
                X, y,
                sample_weight,
                random_state)

            # Early termination.
            if sample_weight is None:
                break

            self.estimator_weights_[iboost] = estimator_weight
            self.estimator_errors_[iboost] = estimator_error

            # Stop if error is zero.
            if estimator_error == 0:
                break

            sample_weight_sum = np.sum(sample_weight)

            # Stop if the sum of sample weights has become non-positive.
            if sample_weight_sum <= 0:
                break

            if iboost < self.n_estimators - 1:
                # Normalize.
                sample_weight /= sample_weight_sum

        return self

In [73]:
def obtain_data(dataset_name):
    dataset = fetch_datasets()[dataset_name]
    return dataset.data,dataset.target

def convert_classes(y):
    default_classes = np.unique(y)
#     print("Default classes of the dataset were: ",default_classes)
    maj_class = -1
    min_class = 1
    if sum(y == default_classes[0]) > sum(y == default_classes[1]):
    #     maj_class = default_classes[0]
    #     min_class = default_classes[1]
        y[y==default_classes[0]] = maj_class
        y[y==default_classes[1]] = min_class
    else:
    #     maj_class = default_classes[1]
    #     min_class = default_classes[0]
        y[y==default_classes[1]] = maj_class
        y[y==default_classes[0]] = min_class

#     print("There are {} instances for the majoritary class".format(sum(y == maj_class)))
#     print("There are {} instanes for the minoritary class".format(sum(y == min_class)))
    return [maj_class,min_class], maj_class, min_class

def train(X_train, y_train, method_name, base_classifier, T, min_class, IR):
    if method_name=='adaboost':
        clf = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=T)
    elif method_name=='RUSBoost':
        clf = RUSBoostClassifier(base_estimator=base_classifier,n_estimators=T,sampling_strategy='majority')
    elif method_name=='SMOTEBoost':
        n_syn = (math.floor(IR)-1)*X_train[y_train==min_class].shape[0]
        clf = SMOTEBoost(n_samples=n_syn, base_estimator=base_classifier, n_estimators=T)
    elif method_name=='DERSBoost':
        clf = DERSBoost(base_estimator=base_classifier, n_estimators=T)

    clf.fit(X_train,y_train)
    return clf

def gmean_test(clf, X_test, y_test):
    # Se calcula el porcentaje de acierto de AdaBoost
    acc = clf.score(X_test,y_test)*100
#     accGlobal.append(acc)
    y_pred = clf.predict(X_test)
    gmean = geometric_mean_score(y_test, y_pred)*100
#     gmeanGlobal.append(gmean)
    bAcc = balanced_accuracy_score(y_test, y_pred)*100
#     baccGlobal.append(bAcc)
    
    return gmean

def train_ensemble_method(dataset_name,method_name, T=10, k=5):
    #fetch data from dataset
    X, y = obtain_data(dataset_name)
    print("Dataset of size {}".format(X.shape))
    
    #convert, just in case, class labels to -1 (majoritary class) and 1 (minoritari class)
    classes, maj_class, min_class = convert_classes(y)
    
    #number of instances of each class and IR
    n_maj = X[y==maj_class].shape[0]
    n_min = X[y==min_class].shape[0]
    IR = n_maj/n_min
    print("There are {} instances for the majoritary class".format(n_maj))
    print("There are {} instanes for the minoritary class".format(n_min))
    print("IR of the dataset: ",IR)
    
    # Llamada al constructor del clasificador 
    dtc = DecisionTreeClassifier(criterion='entropy', max_depth=1)

    kf = StratifiedKFold(n_splits=k)

    accGlobal = []
    gmean = []
    baccGlobal = []
    for train_index, test_index in kf.split(X,y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf = train(X_train, y_train, method_name, dtc, T, min_class, IR)
        
        partial_gmean = gmean_test(clf, X_test, y_test)
        
        gmean.append(partial_gmean)
        
    print(gmean)
    rend = stats.mean(gmean)
    print("Rendimiento del clasificador {}: {}".format(method_name,rend))
    return rend, IR
    

In [74]:
imblearn_datasets = [
    'ecoli',
    'optical_digits',
    'satimage',
    'pen_digits',
    'abalone',
    'sick_euthyroid',
    'spectrometer',
    'car_eval_34',
    'isolet',
    'us_crime',
    'yeast_ml8',
    'scene',
    'libras_move',
    'thyroid_sick',
    'coil_2000',
    'arrhythmia',
    'solar_flare_m0',
    'oil',
    'car_eval_4',
    'wine_quality',
    'letter_img',
    'yeast_me2',
    'webpage',
    'ozone_level',
    'mammography',
    'protein_homo',
    'abalone_19'
]

In [63]:
r = train_ensemble_method('ecoli','DERSBoost')

Dataset of size (336, 7)
There are 301 instances for the majoritary class
There are 35 instanes for the minoritary class
IR of the dataset:  8.6


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Conjunto de entrenamiento original de tamaño: 268
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 240
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 139
Conjunto de entrenamiento actual de tamaño: 167
Gmean: 0.7655590023509528
Gmean: 0.8172252705693472
Gmean: 0.7367883976130072
Gmean: 0.8456454807244981
Gmean: 0.7399788580489365
Gmean: 0.8913052715585238
Gmean: 0.8050764858994133
Gmean: 0.812403840463596
Gmean: 0.6625891564490792
Gmean: 0.48442987566803336
gen	evals	std    	min    	avg   	max     
0  	10   	0.10903	0.48443	0.7561	0.891305
Gmean: 0.7299963950884314
Gmean: 0.7366380786483473
Gmean: 0.8115026712006891
Gmean: 0.7986099033807292
Gmean: 0.7709910794438827
Gmean: 0.6998542122237651
Gmean: 0.816496580927726
Gmean: 0.8075728530872482
Gmean: 0.8312094145936335
Gmean: 0.7033595010646003
1  	10   	0.0481505	0.70336	0.80657	0.891305
Gmean: 0.7083376033211463
Gmean: 0.816496580927726
Gmean: 0.7372611524884796
Gmean:

/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Gmean: 0.7733088933397007
Gmean: 0.7798635358208024
Gmean: 0.6614378277661477
Gmean: 0.6411794687223782
Gmean: 0.7631205286037525
Gmean: 0.8562894997120268
Gmean: 0.7407971974871922
Gmean: 0.8728715609439696
Gmean: 0.7510155725708372
Gmean: 0.7381398907652061
11 	10   	0.0392211	0.825723	0.889227	0.964486
Gmean: 0.7293872016291667
Gmean: 0.831038506929733
Gmean: 0.7655590023509528
Gmean: 0.8765230646861663
Gmean: 0.820412654142367
Gmean: 0.8541614816500999
Gmean: 0.7905694150420948
Gmean: 0.8129201441957019
Gmean: 0.7626009990079631
Gmean: 0.8187261446906408
12 	10   	0.0383839	0.831039	0.889759	0.964486
Gmean: 0.7905694150420949
Gmean: 0.9085135251589959
Gmean: 0.7407971974871922
Gmean: 0.8173225766684681
Gmean: 0.8025005642584075
Gmean: 0.8150678894028793
Gmean: 0.7750157099371281
Gmean: 0.8817077961965372
Gmean: 0.7822327734684298
Gmean: 0.8042718115495202
13 	10   	0.0332215	0.83666 	0.897506	0.964486
Gmean: 0.9294016222362012
Gmean: 0.7669649888473704
Gmean: 0.914991421995628
Gmea

...

/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Gmean: 0.9073893771295621
Gmean: 0.9316997993608415
Gmean: 0.9309493362512627
Gmean: 0.959397030925551
Gmean: 0.9639912695137784
Gmean: 0.9402152508000778
Gmean: 0.8744746321952062
Gmean: 0.9079666047754861
Gmean: 0.8467500662723054
Gmean: 0.924270608979851
gen	evals	std      	min    	avg    	max     
0  	10   	0.0344911	0.84675	0.91871	0.963991
Gmean: 0.899673609101232
Gmean: 0.9703884539097978
Gmean: 0.9431257867152009
Gmean: 0.9444164468894736
Gmean: 0.9662081918411389
Gmean: 0.8826115764526199
Gmean: 0.9117197673942958
Gmean: 0.9252167630517304
Gmean: 0.9867792360446105
Gmean: 0.9630434402528858
1  	10   	0.0250149	0.907389	0.947348	0.986779
Gmean: 0.9845553501483104
Gmean: 0.9213798402227212
Gmean: 0.9245905904654498
Gmean: 0.9565078304365836
Gmean: 0.9361756080885009
Gmean: 0.9734425311469512
Gmean: 0.9638062761761291
Gmean: 0.9085963807262019
Gmean: 0.8692269873603532
Gmean: 0.9074852129730301
2  	10   	0.017426 	0.925217	0.963596	0.986779
Gmean: 0.9355603381115871
Gmean: 0.9508

Gmean: 0.9814213607615716
Gmean: 0.9580457450241959
26 	10   	0.00895609	0.966208	0.982369	0.99969 
Gmean: 0.9719864050705616
Gmean: 0.9388899754961486
Gmean: 0.9778803607803973
Gmean: 0.9021347558954185
Gmean: 0.9555330859059091
Gmean: 0.9141316021452747
Gmean: 0.9165300178459949
Gmean: 0.9128709291752769
Gmean: 0.9049840666859806
Gmean: 0.9774414389667097
27 	10   	0.00895609	0.966208	0.982369	0.99969 
Gmean: 0.948129968579089
Gmean: 0.9671026887665389
Gmean: 0.9546451813444093
Gmean: 0.902109066483585
Gmean: 0.9673619971740891
Gmean: 0.8892222930823348
Gmean: 0.9202915399314313
Gmean: 0.9419296920592422
Gmean: 0.973187044332082
Gmean: 0.9314032073734826
28 	10   	0.00875225	0.967362	0.982485	0.99969 
Gmean: 0.9531763930544696
Gmean: 0.922974334537046
Gmean: 0.9362188283701055
Gmean: 0.9630409805248442
Gmean: 0.9537350297287273
Gmean: 0.9644261421571783
Gmean: 0.8874277264357097
Gmean: 0.9990701997059669
Gmean: 0.8317940817700024
Gmean: 0.9034785677568513
29 	10   	0.0100175 	0.96736

Gmean: 0.9364812998998276
Gmean: 0.9496762821877026
Gmean: 0.9402499662597725
53 	10   	0.0093786 	0.973443	0.989694	0.99969 
Gmean: 0.9190575372593898
Gmean: 0.9088505734084537
Gmean: 0.9380240246906879
Gmean: 0.9735201906024749
Gmean: 0.9195897673618854
Gmean: 0.8933243210477803
Gmean: 0.9348431012376482
Gmean: 0.8542421961772491
Gmean: 0.9438235485528483
Gmean: 0.9252564345149609
54 	10   	0.0093786 	0.973443	0.989694	0.99969 
Gmean: 0.9731324224303965
Gmean: 0.9354143466934853
Gmean: 0.896024625631374
Gmean: 0.8858774811251418
Gmean: 0.9294142210774262
Gmean: 0.9397440417753975
Gmean: 0.9316078236565343
Gmean: 0.9265030047088187
Gmean: 0.9438026847859685
Gmean: 0.9653072991634227
55 	10   	0.0093786 	0.973443	0.989694	0.99969 
Gmean: 0.9126036561702289
Gmean: 0.999404229914351
Gmean: 0.9403015198825015
Gmean: 0.8905186408242552
Gmean: 0.9621325213800873
Gmean: 0.9737201193255615
Gmean: 0.9868211738425042
Gmean: 0.912594761893454
Gmean: 0.9340045277359876
Gmean: 0.945345429955152
56

Gmean: 0.91471561447708
Gmean: 0.9204903118802398
Gmean: 0.9483969895188538
Gmean: 0.9392241038507154
80 	10   	0.0052357 	0.987108	0.994721	0.999692
Gmean: 0.9392134569377365
Gmean: 0.9546728909055322
Gmean: 0.9255428666177289
Gmean: 0.9147845763926178
Gmean: 0.9842839689721424
Gmean: 0.9195930510282038
Gmean: 0.9987767579519153
Gmean: 0.8886552004654004
Gmean: 0.9500492155359249
Gmean: 0.9301000588937754
81 	10   	0.0052357 	0.987108	0.994721	0.999692
Gmean: 0.9475512173393144
Gmean: 0.9599076327221007
Gmean: 0.9158946901192466
Gmean: 0.9074774360504022
Gmean: 0.9039758516167389
Gmean: 0.9526042581585656
Gmean: 0.9662504037103177
Gmean: 0.9438192909103336
Gmean: 0.9601951581195535
Gmean: 0.9029313183520137
82 	10   	0.0052357 	0.987108	0.994721	0.999692
Gmean: 0.9288497456509813
Gmean: 0.9876499904486613
Gmean: 0.9500367735376395
Gmean: 0.9394988701159014
Gmean: 0.963605772808117
Gmean: 0.9039889694875116
Gmean: 0.9047156175902409
Gmean: 0.9652017858539179
Gmean: 0.9032362120547526
G

In [76]:
train_ensemble_method('spectrometer','DERSBoost')
train_ensemble_method('libras_move','DERSBoost')
train_ensemble_method('arrhythmia','DERSBoost')

Dataset of size (531, 93)
There are 486 instances for the majoritary class
There are 45 instanes for the minoritary class
IR of the dataset:  10.8


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Conjunto de entrenamiento original de tamaño: 424
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 388
 	 nº de ejemplos clase MINORITARIA: 36
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 264
Conjunto de entrenamiento actual de tamaño: 300


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

Conjunto de entrenamiento original de tamaño: 425
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 389
 	 nº de ejemplos clase MINORITARIA: 36
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 272
Conjunto de entrenamiento actual de tamaño: 308


...

/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

Conjunto de entrenamiento original de tamaño: 362
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 342
 	 nº de ejemplos clase MINORITARIA: 20
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 298
Conjunto de entrenamiento actual de tamaño: 318


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

[88.39656943780167, 99.41690465022816, 87.85013977691514, 88.38418809879035, 76.54295218641468]
Rendimiento del clasificador DERSBoost: 88.11815083003


(88.11815083003, 17.08)

In [75]:
rendimientos = []
IRlista = []
for dataset in imblearn_datasets:
    print(dataset.upper())
    r, IR = train_ensemble_method(dataset,'DERSBoost')
    rendimientos.append(r)
    IRlista.append(IR)
    print()


ECOLI
Dataset of size (336, 7)
There are 301 instances for the majoritary class
There are 35 instanes for the minoritary class
IR of the dataset:  8.6


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Conjunto de entrenamiento original de tamaño: 268
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 240
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 140
Conjunto de entrenamiento actual de tamaño: 168


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

Conjunto de entrenamiento original de tamaño: 269
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 241
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 140
Conjunto de entrenamiento actual de tamaño: 168


...

Conjunto de entrenamiento original de tamaño: 269
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 241
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 141
Conjunto de entrenamiento actual de tamaño: 169


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

[100.0, 100.0, 80.6225774829855, 67.25927091345493, 91.80725150319788]
Rendimiento del clasificador DERSBoost: 87.93781997992767

OPTICAL_DIGITS
Dataset of size (5620, 64)
There are 5066 instances for the majoritary class
There are 554 instanes for the minoritary class
IR of the dataset:  9.144404332129964


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


KeyboardInterrupt: 

In [ ]:
print("Lista de rendimientos para cada uno de los datasets: ",rendimientos)
print()
# print(rendimientos.index(max(rendimientos)))

rend_sorted_idx = np.argsort(rendimientos)
# print(rend_sorted_idx[-5:])
best_rend_values = np.array(rendimientos)[rend_sorted_idx[-5:]]
best_rend_datasets = np.array(imblearn_datasets)[rend_sorted_idx[-5:]]
best_rend_IR = np.array(IRlista)[rend_sorted_idx[-5:]]
print("Top 5 rendimientos: ",best_rend_values)
print()
print("Datasets correspondientes a los top 5 rendimientos: ",best_rend_datasets)
print()
# print("IR correspondiente a los top 5 rendimientos: ",best_rend_IR)
print()


worst_rend_values = np.array(rendimientos)[rend_sorted_idx[:5]]
worst_rend_datasets = np.array(imblearn_datasets)[rend_sorted_idx[:5]]
worst_rend_IR = np.array(IRlista)[rend_sorted_idx[:5]]
print("Peores 5 rendimientos: ",worst_rend_values)
print()
print("Datasets correspondientes a los 5 peores rendimientos: ",worst_rend_datasets)
print()
# print("IR correspondiente a los 5 peores rendimientos: ",worst_rend_IR)

No existe relación entre el IR ofrecido por el dataset y el rendimiento del clasificador

In [ ]:
print("La media de rendimiento entre todos los datasets es de :",stats.mean(rendimientos))

In [ ]:
print("n_samples default\nLa media de rendimiento entre todos los datasets es de :",stats.mean(rendimientos))

In [ ]:
print("n_samples n+*(floor(ir)-1)\nLa media de rendimiento entre todos los datasets es de :",stats.mean(rendimientos))